In [ ]:
!pip install -U --quiet gluoncv

In [ ]:
import gluoncv
import os

mod_dir = os.getcwd() + '/model'

if not os.path.exists(mod_dir):
    os.makedirs(mod_dir)

net = gluoncv.model_zoo.get_model('faster_rcnn_fpn_resnet101_v1d_coco', pretrained=True, root=mod_dir)

In [ ]:
import tarfile

def flatten(tarinfo):
    tarinfo.name = os.path.basename(tarinfo.name)
    return tarinfo
    
tar = tarfile.open("model.tar.gz", "w:gz")
tar.add("model", filter=flatten)
tar.close()

In [ ]:
import sagemaker
from sagemaker.mxnet.model import MXNetModel
from sagemaker import get_execution_role
from time import gmtime, strftime

role = get_execution_role()

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')
endpoint_name = 'GluonCV-Demo-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

In [ ]:
sagemaker_model = MXNetModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                             role = role,
                             py_version='py3',
                             framework_version='1.4.1',
                             entry_point = 'rcnn.py')

In [ ]:
predictor = sagemaker_model.deploy(initial_instance_count=1,instance_type='ml.p2.xlarge',endpoint_name=endpoint_name)

In [ ]:
#Download a Test Image
im_fname = gluoncv.utils.download('https://github.com/dmlc/web-data/blob/master/' +
                          'gluoncv/detection/biking.jpg?raw=true', path='biking.jpg')

In [ ]:
import boto3
import json
import cv2
import numpy as np

sm_runtime = boto3.Session().client('sagemaker-runtime')

with open('biking.jpg', 'rb') as f:
    payload = f.read()
    
output = sm_runtime.invoke_endpoint(EndpointName=endpoint_name,
                                      ContentType='application/x-image',
                                      Body=payload)

result = json.loads(output['Body'].read().decode())
ax = gluoncv.utils.viz.plot_bbox(cv2.imread('biking.jpg'), result['bboxes'], np.asarray(result['scores']), np.asarray(result['box_ids']), 
                         class_names=result['classes'])